# Laboratório Prático: Textos

**O Problema do Mundo Real:**
Queremos criar um sistema que leia comentários de clientes e diga se eles estão Satisfeitos (Positivo) ou Insatisfeitos (Negativo).
O computador não entende português. Ele não sabe o que é a palavra "Fantástico". Ele só entende números. A nossa missão é traduzir texto para números e alimentar a nossa Rede Neuronal.

Vamos começar por importar as nossas ferramentas.

**O que vamos importar?**
* `torch`: A biblioteca principal do PyTorch. É como a caixa de ferramentas geral.
* `torch.nn`: O módulo de Redes Neuronais (*Neural Networks*). Contém os blocos de construção (camadas, funções de ativação, função de perda).
* `torch.optim`: O módulo de Otimizadores. Contém a nossa "bússola" para descer a montanha do erro.
* `CountVectorizer` (do *scikit-learn*): O nosso tradutor. Ele vai pegar no texto e transformá-lo num "Saco de Palavras" (uma lista de contagens numéricas).

In [1]:
# Importação das bibliotecas
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer

## 1. Os Nossos Dados

Vamos criar um pequeno conjunto de dados de exemplo. Temos uma lista de frases e uma lista de rótulos (a resposta certa).

**A Regra dos Rótulos:**
* `1` = Sentimento Positivo
* `0` = Sentimento Negativo

Em Machine Learning, a rede vai olhar para o texto, tentar adivinhar, e comparar o seu palpite com o rótulo real para aprender com o erro.

In [14]:
# Textos (Comentários de clientes)
comentarios = [
    "Adorei o produto, qualidade fantástica",   # Positivo
    "Péssimo serviço, odiei a experiência",     # Negativo
    "Muito bom, recomendo a toda a gente",      # Positivo
    "Dinheiro deitado à rua, não funciona",     # Negativo
    "Excelente, superou as minhas expectativas",# Positivo
    "Horrível, nunca mais volto a comprar",     # Negativo
    "Nada a reclamar, vou voltar"               # Positivo
]

# Rótulos (O gabarito: 1 = Positivo, 0 = Negativo)
rotulos = [1, 0, 1, 0, 1, 0, 1]

print("Dados carregados com sucesso!")

Dados carregados com sucesso!


## 2. A Tradução: De Texto para Números (Bag of Words)

Agora vamos usar o `CountVectorizer`. Ele funciona assim:
1. Lê todos os comentários e cria um **Vocabulário** (uma lista com todas as palavras únicas que apareceram, ignorando maiúsculas e pontuação).
2. Transforma cada frase numa lista de números. Cada número diz quantas vezes uma determinada palavra do vocabulário apareceu naquela frase.

**Parâmetros e Funções utilizadas:**
* `vetorizador.fit_transform(dados)`: Faz duas coisas ao mesmo tempo. O `fit` estuda o texto e constrói o dicionário. O `transform` aplica a tradução matemática.
* `.toarray()`: Converte o resultado estranho do scikit-learn numa grelha (matriz) normal de números.
* `torch.tensor(dados, dtype=...)`: Converte os números do Python para o formato oficial do PyTorch (Tensores, que podem ir para a Placa Gráfica). O `dtype=torch.float32` indica números com casas decimais (que a rede adora), e `torch.long` indica números inteiros pesados (obrigatório para os rótulos de classe).

In [15]:
# Inicializar o tradutor
vetorizador = CountVectorizer()

# Aprender o vocabulário e traduzir as frases
X_numerico = vetorizador.fit_transform(comentarios).toarray()

# Vamos descobrir quantas palavras únicas o nosso dicionário aprendeu
tamanho_vocabulario = X_numerico.shape[1]
print(f"O nosso dicionário tem {tamanho_vocabulario} palavras únicas.")

# Converter para o formato de Tensores do PyTorch
X_tensor = torch.tensor(X_numerico, dtype=torch.float32)
y_tensor = torch.tensor(rotulos, dtype=torch.long)

# Mostrando o resultado do primeiro comentário convertido
print(f"\nPrimeiro comentário original: '{comentarios[0]}'")
print(f"Primeiro comentário em números: {X_tensor[0]}")

O nosso dicionário tem 32 palavras únicas.

Primeiro comentário original: 'Adorei o produto, qualidade fantástica'
Primeiro comentário em números: tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


## 3. Construindo a Rede Neuronal (A Fábrica)

Chegou a hora de montar a nossa rede usando a ferramenta `nn.Sequential`. Pense nisto como uma caixa que contém uma linha de montagem. O dado entra no topo e vai passando camada a camada.

**As peças da nossa arquitetura:**
* `nn.Linear(in_features, out_features)`: É o nosso grupo de "especialistas".
  * `in_features`: Quantas informações entram? Tem de ser EXATAMENTE igual ao número de palavras do nosso vocabulário. Se a nossa "receção" tem 26 palavras, precisamos de 26 balcões de entrada.
  * `out_features`: O número de especialistas que queremos nesta camada. Aqui escolhemos 8 (um hiperparâmetro, podíamos ter escolhido 16 ou 32).
* `nn.ReLU()`: A Função de Ativação. É o "tempero" matemático que zera os números negativos, permitindo à rede aprender padrões complexos (curvas) em vez de apenas linhas retas.
* A Última Camada `nn.Linear(8, 2)`: Recebe a informação mastigada pelos 8 especialistas e resume tudo em apenas 2 respostas finais (Pontuação para "Negativo" e Pontuação para "Positivo").

In [16]:
# Definindo a arquitetura
modelo = nn.Sequential(
    # Camada de Entrada -> Camada Oculta
    nn.Linear(in_features=tamanho_vocabulario, out_features=8),

    # Função de Ativação (O filtro Não-Linear)
    nn.ReLU(),

    # Camada Oculta -> Camada de Saída (Decisão)
    nn.Linear(in_features=8, out_features=2)
)

print(modelo)

Sequential(
  (0): Linear(in_features=32, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=2, bias=True)
)


## 4. Preparando o Treino: O Professor e a Bússola

A nossa rede nasceu agora, os seus pesos (as conexões matemáticas) são todos aleatórios. Ela não sabe nada. Precisamos treiná-la.

**As ferramentas de treino:**
* `nn.CrossEntropyLoss()`: É o "Professor" (Função de Perda). Ele compara a resposta da rede com o rótulo verdadeiro e grita: "Estás muito longe do resultado certo!". Ele usa internamente a fórmula *Softmax* para problemas de classificação.
* `optim.Adam(params, lr)`: A "Bússola" (Otimizador). Baseado no erro que o professor gritou, o Adam calcula como rodar os "botões" (pesos) da rede para diminuir esse erro.
  * `params=modelo.parameters()`: Dizemos à bússola para controlar todos os pesos da nossa rede.
  * `lr=0.05`: É a *Learning Rate* (Taxa de Aprendizado). Indica o tamanho do passo que vamos dar ao descer a montanha do erro. Se for muito grande, tropeçamos; se for muito pequeno, não saímos do sítio.

In [17]:
# O Professor (Calcula o Erro)
funcao_perda = nn.CrossEntropyLoss()

# A Bússola (Ajusta os Pesos)
otimizador = optim.Adam(params=modelo.parameters(), lr=0.05)

## 5. O Ciclo de Treino (The Training Loop)

A rede aprende por repetição. Uma **Época (Epoch)** ocorre quando a rede lê TODO o conjunto de dados uma vez. Vamos treinar por 50 épocas.

**Os 5 Passos Mágicos do PyTorch (dentro do loop):**
1. `otimizador.zero_grad()`: Limpa o quadro branco. Apaga a memória de quem teve a culpa no erro anterior.
2. `modelo(X_tensor)`: O *Forward Pass*. A rede tenta adivinhar a resposta baseada no que sabe agora.
3. `funcao_perda(previsoes, y_tensor)`: Calcula a distância (Loss) entre o palpite da rede e a verdade.
4. `erro.backward()`: O *Backpropagation*. O momento "Apontar o dedo". A matemática volta de trás para a frente a calcular a culpa (gradiente) de cada peso no erro total.
5. `otimizador.step()`: A ação. A bússola diz: "Já que a culpa foi deste peso, vamos diminuí-lo um pouco". A rede acaba de ficar ligeiramente mais inteligente.

In [18]:
epocas = 50

print("A iniciar o treino...\n")

for epoca in range(epocas):

    # 1. Limpar gradientes
    otimizador.zero_grad()

    # 2. Fazer previsões
    previsoes = modelo(X_tensor)

    # 3. Calcular o erro (Loss)
    erro = funcao_perda(previsoes, y_tensor)

    # 4. Calcular as culpas (Backpropagation)
    erro.backward()

    # 5. Atualizar os pesos (Otimizar)
    otimizador.step()

    # Imprimir o progresso a cada 10 épocas
    if (epoca + 1) % 10 == 0:
        print(f"Época {epoca+1}/50 | Erro atualizado: {erro.item():.4f}")

print("\nTreino finalizado! A rede decorou a montanha do erro.")

A iniciar o treino...

Época 10/50 | Erro atualizado: 0.1008
Época 20/50 | Erro atualizado: 0.0008
Época 30/50 | Erro atualizado: 0.0000
Época 40/50 | Erro atualizado: 0.0000
Época 50/50 | Erro atualizado: 0.0000

Treino finalizado! A rede decorou a montanha do erro.


## 6. O Teste Final: O Computador Aprendeu a Ler?

Vamos escrever duas frases que a rede **NUNCA** viu na vida. Será que ela consegue generalizar o aprendizado e adivinhar o sentimento?

**Passos importantes:**
* `vetorizador.transform(textos)`: Note que aqui **NÃO usamos o "fit"**, apenas o "transform". O nosso dicionário já está fechado. Apenas traduzimos o texto novo usando as regras antigas.
* `with torch.no_grad():`: É um aviso ao PyTorch: "Desliga o cálculo de gradientes. Estamos apenas a fazer um teste, não quero que aprendas nem gastes memória agora."
* `torch.softmax(saidas, dim=1)`: Pega na pontuação bruta que a rede cospe e transforma-a numa percentagem de certeza (0% a 100%), onde a soma das classes dá 1.
* `torch.argmax(probabilidades, dim=1)`: Olha para as duas percentagens e devolve o índice daquela que ganhou (Ex: "A classe 1 teve 90%, então o argmax responde '1'").

In [19]:
# Frases novas de teste
frases_teste = [
    "Achei o produto fantástico e excelente", # Esperamos Positivo (1)
    "Serviço horrível e péssimo",             # Esperamos Negativo (0)
    "Nada a reclamar, vou voltar"             # Esperamos Positivo (1)
]

# Traduzir as novas frases usando o dicionário que já treinamos
X_teste = vetorizador.transform(frases_teste).toarray()
X_teste_tensor = torch.tensor(X_teste, dtype=torch.float32)

print("A fazer previsões em dados novos...\n")

# Desligar o "modo de aprendizagem"
with torch.no_grad():
    # Passar os dados pela rede
    saidas_brutas = modelo(X_teste_tensor)

    # Converter pontuações brutas em Probabilidades (%)
    probabilidades = torch.softmax(saidas_brutas, dim=1)

    # Escolher a classe com maior probabilidade
    classes_previstas = torch.argmax(probabilidades, dim=1)

# Imprimir o resultado de forma amigável para humanos
for i in range(len(frases_teste)):
    if classes_previstas[i].item() == 1:
        sentimento = "Positivo"
    else:
        sentimento = "Negativo"

    # Extrair a confiança percentual da classe vencedora
    confianca = probabilidades[i][classes_previstas[i]].item() * 100

    print(f"Comentário: '{frases_teste[i]}'")
    print(f" -> O modelo diz que é: {sentimento} (Certeza: {confianca:.1f}%)\n")

A fazer previsões em dados novos...

Comentário: 'Achei o produto fantástico e excelente'
 -> O modelo diz que é: Positivo (Certeza: 100.0%)

Comentário: 'Serviço horrível e péssimo'
 -> O modelo diz que é: Negativo (Certeza: 100.0%)

Comentário: 'Nada a reclamar, vou voltar'
 -> O modelo diz que é: Positivo (Certeza: 100.0%)

